# Design a bigger dataset of queries by using data augmentation

We want to change two things:
*  we change some filter conditions 
*  we (exhaustively) change the attribute in the MIN-aggregation, such that for each query one variable of each relation is represented once in the new dataset

Dataset size in the beginning: 229
*  STATS: 146
*  SNAP: 40
*  JOB: 15
*  LSQB: 2
*  HETIO: 26

#### Augmentation: Filter (+ changing connections for HETIO)
By hand we change each STATS and JOB query twice with changing different filters. (SNAP and LSQB do not have any filters.)   
(The new queries are called "query"-augF1 and "query"-augF2, where "query" is the original name of the query.)   
For 6 STATS queries there is only one filter. Then we only create one new query "query"-augF1. This is the case for:
*  STATS: 024-017
*  STATS: 025-001
*  STATS: 096-095
*  STATS: 100-005
*  STATS: 111-056
*  STATS: 143-126
In the most cases we change the values of $>, <, \geq, \leq$ conditions.
In some cases we also change the equality conditions and we got the new string values like the following:

![Example Image](images/imdb_random_keyword.PNG)

For the HETIO dataset we create 12 new queries, where we have 4 queries with filters (queries 5-8). Here we again change the filters two times.  
For the other 8 queries we do augmentation in a way that we replace one connection in the graph with another one between the same two nodes.   
(e.g. we replace upregulates with downregulates between disease and gene) 

![Example Image2](images/hetio_graph.PNG)

We get multiple variants for those 8 queries:
*  2 variants for queries: 12, 13
*  3 variants for queries: 10, 11, 14, 16
*  9 variants for queries: 9, 15

Therefore we get a new dataset with size: 591
*  STATS: 140 * 3 + 6 * 2 = 432
*  SNAP: 40
*  JOB: 15 * 3 = 45
*  LSQB: 2
*  HETIO: 26 (not changed) + 4 * 3 (filter) + 2 * 2 + 4 * 3 + 2 * 9 = 72

#### Augmentation: Change aggregate-attribute
Now we take this new dataset and change the attribute in the MIN such that for each relation per query there is a query with an attribute of this table in the aggregation. The new dataset size is now the sum of the number of relations per query.   

In [5]:
# get the number of queries, which we get after this augmentation
import re

# Define input and output file paths
input_file = 'scala_commands_augment_filter.txt'

queries_stats = 0
queries_snap = 0
queries_job = 0
queries_lsqb = 0
queries_hetio = 0

# Open input and output files
with open(input_file, 'r') as f_input:
    # Read input file line by line
    for line in f_input:
        # Split each line into components
        pattern = r'(?<!\\)\"|\"(?<!\\)(?=\s+\"|$)'
        components = re.split(pattern, line)
        
        # Extract relevant information
        benchmark = components[3]
        number = components[5]
        query = components[1].strip()

        query_upper = query.upper()
        from_index = query_upper.find("FROM")
        where_index = query_upper.find("WHERE")
        number_of_relations = query[from_index:where_index].count(",") + 1

        if benchmark == "STATS":
            queries_stats += number_of_relations
        elif benchmark == "SNAP":
            queries_snap += number_of_relations
        elif benchmark == "JOB":
            queries_job += number_of_relations
        elif benchmark == "LSQB":
            queries_lsqb += number_of_relations
        elif benchmark == "HETIO":
            queries_hetio += number_of_relations
        else:
            print("other benchmark?")

print("STATS:", queries_stats)
print("SNAP:", queries_snap)
print("JOB:", queries_job)
print("LSQB:", queries_lsqb)
print("HETIO:", queries_hetio)
print("This gives us", queries_stats + queries_snap + queries_job + queries_lsqb + queries_hetio, "queries in total")

STATS: 1876
SNAP: 244
JOB: 264
LSQB: 14
HETIO: 538
This gives us 2936 queries in total


In [1]:
%%bash
pip install psycopg2-binary

In [2]:
import psycopg2
import re

Do the augmentation:

In [3]:
# Define input and output file paths
input_file = 'scala_commands_augment_filter.txt'
output_file = 'scala_commands_augment_filter_agg.txt'

# Open input and output files
with open(input_file, 'r') as f_input, open(output_file, 'w', newline='') as f_output:
    
    # Read input file line by line
    for line in f_input:
        # Split each line into components
        pattern = r'(?<!\\)\"|\"(?<!\\)(?=\s+\"|$)'
        components = re.split(pattern, line)
        
        benchmark = components[3]
        number = components[5]
        query = components[1].strip()
        
        query_upper = query.upper()
        from_index = query_upper.find("FROM")
        where_index = query_upper.find("WHERE")
        relations_list = query[from_index+4:where_index].split(",")
        relations = {relation.strip().rsplit(maxsplit=1)[-1]: relation.strip().split(maxsplit=1)[0] for relation in relations_list}

        # get all relations occuring in the query and their aliases
        min_index = query_upper.find("MIN")
        agg = query[min_index+4:from_index-2].strip().split(".")[0].strip()
        relations = {key: value for key, value in relations.items() if key != agg}
    
        if benchmark == "JOB":
            database = "imdb"
        else:
            database = benchmark.lower()
        conn = psycopg2.connect(
            host="postgres",
            database=database,
            user=database,
            password=database
        )
        cur = conn.cursor()

        # get one column name for each relation
        new_aggs = []
        for key, value in relations.items():
            query_col = f"""SELECT column_name FROM information_schema.columns WHERE table_name = '{value.lower()}'"""
            cur.execute(query_col)
            row = cur.fetchone()[0]
            new_aggs.append(key + "." + row)
        cur.close()
        conn.close()

        # replace the MIN-aggregate with the new agg (one new query for each relation)
        i = 1
        f_output.write(line)
        for new_agg in new_aggs:
            result = re.sub(r'MIN\([^)]*\)', "MIN(" + new_agg + ")", line)
            result = result[:-2] + "-augA" + str(i) + '"' + "\n"
            f_output.write(result)
            i += 1